In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression

In [6]:
xls = pd.ExcelFile('../data/WorldEnergyBalancesHighlights_final.xlsx')

data = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2],usecols="A:C,Z:BB",na_values=['..','c'])

br = data.loc[('Brazil')]
renew = br.loc[('Renewable sources')]
renew_transpose = renew.transpose()

renew_transpose.values().ravel()

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd


#Define função para calcular o MAPE
def mape(y_pred,y_true):

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


#Carrega os dados

data = pd.read_csv('prime.csv',header=0,index_col=0).sort()

x_data = []
y_data = []

# Formata de maneira que cada linha da matriz X seja composta pelos 6 meses anteriores.
for d in range(6,data.shape[0]):

    x = data.iloc[d-6:d].values.ravel()
    y = data.iloc[d].values[0]

    x_data.append(x)
    y_data.append(y)


x_data = np.array(x_data)
y_data = np.array(y_data)




#Listas para armazenar as previsões de cada modelo
y_pred = []
y_pred_last = []
y_pred_ma = []
y_true = []



#Itera pela série temporal treinando um novo modelo a cada mês
end = y_data.shape[0]
for i in range(30,end):

    x_train = x_data[:i,:]
    y_train = y_data[:i]

    x_test = x_data[i,:]
    y_test = y_data[i]


    model = LinearRegression(normalize=True)
    model.fit(x_train,y_train)

    y_pred.append(model.predict(x_test))
    y_pred_last.append(x_test[-1])
    y_pred_ma.append(x_test.mean())
    y_true.append(y_test)


#Transforma as listas em arrays numpy para facilitar os cálculos
y_pred = np.array(y_pred)
y_pred_last = np.array(y_pred_last)
y_pred_ma = np.array(y_pred_ma)
y_true = np.array(y_true)


#Imprime os erros na tela
print ('\nMean Absolute Percentage Error')
print ('MAPE Regressão Linear', mape(y_pred,y_true))
print ('MAPE Último Valor', mape(y_pred_last,y_true))
print ('MAPE Média Móvel', mape(y_pred_ma,y_true))


print ('\nMean Absolute Error')
print ('MAE Regressão Linear', mean_absolute_error(y_pred,y_true))
print ('MAE Último Valor', mean_absolute_error(y_pred_last,y_true))
print ('MAE Média Móvel', mean_absolute_error(y_pred_ma,y_true))